## Setup Notebook
- Julia Env
- Attach User Libraries

In [1]:
using Pkg
project_root = dirname(dirname(@__DIR__))
Pkg.activate(project_root)     
Pkg.instantiate()     

  Activating project at `d:\Universita\repos\TESI\tesi_data\progetto_s_on_f.md\b_spline\functional-data-regression-mip`


In [2]:

using RCall
user_library_path = "C:/Users/User/AppData/Local/R/cache/R/renv/library/functional-data-regression-mip-e3349204/R-4.3/x86_64-w64-mingw32"
@rput user_library_path
R"""
    .libPaths(c(user_library_path,
    .libPaths()))
"""
    

RObject{StrSxp}
[1] "C:/Users/User/AppData/Local/R/cache/R/renv/library/functional-data-regression-mip-e3349204/R-4.3/x86_64-w64-mingw32"
[2] "C:/Users/User/AppData/Local/R/win-library/4.3"                                                                      
[3] "C:/Program Files/R/R-4.3.1/library"                                                                                 


In [5]:

file_mgmt_file_path = joinpath(project_root, "src", "Julia","utils", "file_management.jl")
include(file_mgmt_file_path)



read_model_results

In [6]:
simulation_name = "3_predictors"
model = "l0_limit"
simulation_conf = "2500_500_6_250_0.01_0.001_1000_[100, 100]_1"
has_time_domain = true
simulation_path =   joinpath(project_root, "outputs", "runs", simulation_name, model, simulation_conf, "model_results" * ".txt")
model_params_path = joinpath(project_root, "outputs", "runs", simulation_name, model, simulation_conf, "model_params" * ".csv")


"d:\\Universita\\repos\\TESI\\tesi_data\\progetto_s_on_f.md\\b_spline\\functional-data-regression-mip\\outputs\\runs\\3_predictors\\l0_limit\\2500_500_6_250_0.01_0.001_1000_[100, 100]_1\\model_params.csv"

In [7]:
using CSV
using DataFrames

results = read_model_results(simulation_path);
# read mode_params.csv in Julia from "model_params_path"
df =CSV.read(model_params_path, DataFrame) 




LoadError: SystemError: opening file "d:\\Universita\\repos\\TESI\\tesi_data\\progetto_s_on_f.md\\b_spline\\functional-data-regression-mip\\outputs\\runs\\3_predictors\\l0_limit\\2500_500_6_250_0.01_0.001_1000_[100, 100]_1\\model_results.txt": No such file or directory

In [6]:
results

Dict{Any, Any} with 6 entries:
  "Beta Matrix"          => [0.999997 1.00004 … 0.633811 0.540305; 6.96835e-6 0…
  "True Predictors"      => [1.0; 1.0; 0.0;;]
  "Evaluated Predictors" => [1.0; 1.0; 0.0;;]
  "True Intercept"       => [0.0;;]
  "Evaluated Intercept"  => [0.0;;]
  "Beta Star"            => [1.01426 0.987147 … 0.639002 0.533365; 0.0 0.0 … 1.…

## Load Setting Data

In [7]:
simulation_settings_file = df.SettingName[1]


"default"

In [33]:
load_conf_file_path = joinpath(project_root, "src", "R","generic_simulator","utils", "loader_utilities.R")



@rput simulation_name
@rput simulation_settings_file
@rput load_conf_file_path
R"""
source(load_conf_file_path)
library(here)

# Retrieve time_domains from the param setting
inputs  <- load_simulation_settings(simulation_name, simulation_settings_file)
time_domains_eval <- lapply(inputs$time_domains, function(domain) {
           seq(from = domain[[1]], to = domain[[2]], length.out = inputs$measurements)
       })
inputs$time_domains <- time_domains_eval
# Recreate the basis values 
source(here("src", "R",   "generic_simulator",  "utils" , "basis_utilities.R")) 

basis_objs <- create_basis(inputs$basis_functions, inputs$time_domains, inputs$norder, inputs$predictors)
result <- smooth_betas_generic(beta_funcs,basis_functions,  inputs$time_domains, basis_objs)
basis_values <- result$basis_values
beta_point_values <- result$beta_point_values

"""



RObject{RealSxp}
     [,1]        [,2]      [,3]       [,4]       [,5]       [,6]       [,7]
[1,]    1 0.999987374 0.9999495 0.99988637 0.99979799 0.99968437 0.99954550
[2,]    0 0.005262275 0.0105244 0.01578624 0.02104764 0.02630846 0.03156855
[3,]    0 0.000000000 0.0000000 0.00000000 0.00000000 0.00000000 0.00000000
           [,8]       [,9]      [,10]      [,11]      [,12]      [,13]
[1,] 0.99938139 0.99919205 0.99897747 0.99873767 0.99847265 0.99818241
[2,] 0.03682776 0.04208596 0.04734299 0.05259871 0.05785297 0.06310563
[3,] 0.00000000 0.00000000 0.00000000 0.00000000 0.00000000 0.00000000
          [,14]      [,15]      [,16]      [,17]      [,18]      [,19]
[1,] 0.99786697 0.99752634 0.99716051 0.99676950 0.99635332 0.99591198
[2,] 0.06835654 0.07360556 0.07885255 0.08409734 0.08933981 0.09457981
[3,] 0.00000000 0.00000000 0.00000000 0.00000000 0.00000000 0.00000000
          [,20]     [,21]     [,22]     [,23]     [,24]     [,25]     [,26]
[1,] 0.99544550 0.9949539 0.9944371

In [56]:
# Sample input data
time_domains = rcopy(R"(inputs$time_domains)");

# betas and basis
basis_values = rcopy(R"(basis_values)");

# beta_point_values
beta_point_values = rcopy(R"(beta_point_values)");

In [63]:
plot_file_path = joinpath(project_root, "src", "Julia","utils", "plot.jl")
include(plot_file_path)


output_folder= joinpath(project_root, "outputs", "plots", simulation_name, model, simulation_settings_file,simulation_conf)
γ_matrix = results["Beta Matrix"]
γ_star = results["Beta Star"]
plot_combined_predicted_curve(beta_point_values, γ_star, basis_values,  time_domains, output_folder, true)


LoadError: DimensionMismatch: A has dimensions (200,5) but B has dimensions (1,6)